In [ ]:
# for demo purpose, configure alice and bob in different files
import secrets_alice
import secrets_bob

In [ ]:
import sdk_mind_fhe
import settings
from web3 import Web3

# 1. Bob prepare keys (once and reuse)

In [ ]:
import secrets_bob
web3 = Web3(Web3.HTTPProvider(secrets_bob.WEB3API))
walletAccount_bob = web3.eth.account.from_key(secrets_bob.walletPrivateKey)

SK_2, PK_2 = sdk_mind_fhe.gen_spend_keys()
SK_b, PK_b = sdk_mind_fhe.gen_fhe_keys()

# 2. Bob register public keys (once and reuse)

In [ ]:
contract_ns = web3.eth.contract(address=settings.CONTRACT_ADDRESS_NS, abi=settings.CONTRACT_ABI_NS)

import secrets_bob
web3 = Web3(Web3.HTTPProvider(secrets_bob.WEB3API))
walletAccount_bob = web3.eth.account.from_key(secrets_bob.walletPrivateKey)
walletAddress_bob = walletAccount_bob.address

nonce = web3.eth.get_transaction_count(walletAddress_bob)
txn = contract_ns.functions.setPKNS(PK_2, PK_b).buildTransaction({
    'from': walletAddress_bob,
    'gasPrice': web3.eth.gas_price,
    'nonce': nonce,
})

signed_txn = walletAccount_bob.signTransaction(txn)
txn_hash = web3.eth.send_raw_transaction(signed_txn.rawTransaction)
print(f'NS Transaction sent: {txn_hash.hex()}')

txn_receipt = web3.eth.wait_for_transaction_receipt(txn_hash)
print(f'NS Transaction mined, status: {txn_receipt["status"]}')

# 3. Alice prepare keys (once and reuse)

In [ ]:
import secrets_alice
web3 = Web3(Web3.HTTPProvider(secrets_alice.WEB3API))
walletAccount_alice = web3.eth.account.from_key(secrets_alice.walletPrivateKey)
walletAddress_alice = walletAccount_alice.address

SK_1, PK_1 = sdk_mind_fhe.gen_tx_keys()

# 4. Alice compute SA keys

In [ ]:
# get Bob's NS
contract_ns = web3.eth.contract(address=settings.CONTRACT_ADDRESS_NS, abi=settings.CONTRACT_ABI_NS)
PK_2, PK_b = contract_ns.functions.getPKNS(walletAddress_bob).call()

PK_z = sdk_mind_fhe.get_SA_pk(PK_1, PK_2)
SA = sdk_mind_fhe.pktoaddr(PK_z)

C1 = sdk_mind_fhe.fhe_enc(SK_1, PK_b)

contract_msg = web3.eth.contract(address=settings.CONTRACT_ADDRESS_MSG, abi=settings.CONTRACT_ABI_MSG)

import secrets_bob
web3 = Web3(Web3.HTTPProvider(secrets_alice.WEB3API))
walletAccount_alice = web3.eth.account.from_key(secrets_alice.walletPrivateKey)
walletAddress_alice = walletAccount_alice.address

nonce = web3.eth.get_transaction_count(walletAddress_alice)
txn = contract_msg.functions.setMSG(C1).buildTransaction({
    'from': walletAddress_alice,
    'gasPrice': web3.eth.gas_price,
    'nonce': nonce,
})

signed_txn = walletAccount_alice.signTransaction(txn)
txn_hash = web3.eth.send_raw_transaction(signed_txn.rawTransaction)
print(f'MSG Transaction sent: {txn_hash.hex()}')

txn_receipt = web3.eth.wait_for_transaction_receipt(txn_hash)
print(f'MSG Transaction mined, status: {txn_receipt["status"]}')

# 5. Alice transfer X of tokens to SA, ERC20 or ETH

In [ ]:
X = 1 # or other amount of eth in this example

txn = dict(
    nonce=Web3.eth.get_transaction_count(Web3.eth.coinbase),
    from=walletAddress_alice,
    to=SA,
    value=X
)

signed_txn = walletAccount_alice.signTransaction(txn)
txn_hash = web3.eth.send_raw_transaction(signed_txn.rawTransaction)
print(f'TX Transaction sent: {txn_hash.hex()}')

txn_receipt = web3.eth.wait_for_transaction_receipt(txn_hash)
print(f'TX Transaction mined, status: {txn_receipt["status"]}')

# 6. Bob scan to target on SA with C1

In [ ]:
C2 = sdk_mind_fhe.fhe_enc(SK_2, PK_b)

contract_msg = web3.eth.contract(address=settings.CONTRACT_ADDRESS_MSG, abi=settings.CONTRACT_ABI_MSG)
C1 = contract_msg.functions.getMSG(walletAddress_alice).call()

C = C1 + C2
SK_z1 = sdk_mind_fhe.fhe_dec(C, SK_b)
PK_z1 = sdk_mind_fhe.sktopk(SK_z1)
SA1 = sdk_mind_fhe.pktoaddr(PK_z1)

if SA != SA1:
    print("# no validated SA and continue")
    pass 
else:
    print("go step 7 to control or withdraw X from SA with SK_z1")
    pass

# 7. Bob manage SA with secret key

In [ ]:
print("more steps to go")